# -1_ Task One: review data insights

This notebook includes some code to collect review data from the `SkyTrax` [https://www.airlinequality.com] website using `BeautifulSoup` package for web scrapping. The collected data are saved it into a local `.csv` to perform analysis.

## -1.a/b- Data scrapping & cleaning

For this task, we focus our analysis on reviews related to British Airways and the Airline itself.
If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. 
Now, we use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [77]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import matplotlib


In [78]:
bads_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 100
# if you want to collect more data, try increasing the number of pages!
page_size = 100

reviews = []

# loop to collect 1000 reviews by iterating through the paginated pages
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{bads_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"
    print(url)
    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    # loop to extract the "text_content" HTML class data
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        # clean this data to remove any unnecessary text from each of the rows.
        # Delete "✅ Trip Verified" or "Not Verified as it's not relevant for investigation.
        # Begin all review after the "|" sequence, after strimming the spaces " ".
        review = para.get_text()
        reviews.append(review[review.index("|")+1:].lstrip())

    print(f"   ---> {len(reviews)} total reviews")


Scraping page 1
https://www.airlinequality.com/airline-reviews/british-airways/page/1/?sortby=post_date%3ADesc&pagesize=100
   ---> 100 total reviews
Scraping page 2
https://www.airlinequality.com/airline-reviews/british-airways/page/2/?sortby=post_date%3ADesc&pagesize=100
   ---> 200 total reviews
Scraping page 3
https://www.airlinequality.com/airline-reviews/british-airways/page/3/?sortby=post_date%3ADesc&pagesize=100
   ---> 300 total reviews
Scraping page 4
https://www.airlinequality.com/airline-reviews/british-airways/page/4/?sortby=post_date%3ADesc&pagesize=100
   ---> 400 total reviews
Scraping page 5
https://www.airlinequality.com/airline-reviews/british-airways/page/5/?sortby=post_date%3ADesc&pagesize=100
   ---> 500 total reviews
Scraping page 6
https://www.airlinequality.com/airline-reviews/british-airways/page/6/?sortby=post_date%3ADesc&pagesize=100
   ---> 600 total reviews
Scraping page 7
https://www.airlinequality.com/airline-reviews/british-airways/page/7/?sortby=post_d

ValueError: substring not found

In [ ]:
# Transform data to DataFrame format
df = pd.DataFrame()
df["reviews"] = reviews
df.head()


,reviews
0,Extremely rude ground service. We were non-rev...
1,My son and I flew to Geneva last Sunday for a ...
2,For the price paid (bought during a sale) it w...
3,Flight left on time and arrived over half an h...
4,"Very Poor Business class product, BA is not ev..."


In [ ]:
# Store DataFrame to CSV format
df.to_csv("data/BADS_reviews.csv")


Congratulations! Now we have our dataset for this task! 


## -1.b- Data analysing
### -1.b.1- Sentiment analysis - Word embedding
For this task, we focus our data analysing on Natural Language Processing (NLP)  with sentiment analysing.
As working on words directly is not possible we need to realize a word embedding.
First we have to vextorize words to provide a mapping from the dictionary of words to vectors of a fixed dimension.
For this task, we use `Python` and `word2vec` library to vectorize review words.

# Keras Embedding
Thanks to the `Embedding` layer in `Keras` library to feed a Recurrent Neural Network with vectorized text!

In [ ]:
## Requirement

Install [TensorFlow Datasets](https://www.tensorflow.org/datasets):

```bash
pip install tensorflow-datasets
```


In [ ]:

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

### load the data ###

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)

    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)

        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]

        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]

    # Decode each word in UTF-8
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)


In [ ]:
# lenght of datasets
len(X_train), len(y_train), len(X_test), len(y_test)


In [ ]:
 # Ex. of sentences X => y:
[[ f'X_train[{i}][{X_train[i][j]}] => y_train = {y_train[i]}' for j in range(len(X_train[i])//30)] for i in range(555, 560)]
